# 网络模型、正向传播函数

In [1]:
# 导入软件包
import pandas as pd
from math import sqrt as sqrt
from itertools import product as product

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function


In [2]:
#创建34层神经网络的vgg模块
def make_vgg():
    layers = []
    in_channels = 3

    #在vgg模块中使用的卷积层和最大池化等的通道数
    cfg = [64,64,'M',128,128,'M',256,256,256,'MC',512,512,512,'M',512,512,512]

    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
        elif v == 'MC':
            layers += [nn.MaxPool2d(2,2,ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels,v,kernel_size=3,padding=1)
            layers += [conv2d,nn.ReLU(inplace=True)]
            in_channels = v 
    pool5 = nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
    conv6 = nn.Conv2d(512,1024,kernel_size=3,padding=6,dilation=6)
    conv7 = nn.Conv2d(1024,1024,kernel_size=1)
    layers += [pool5, conv6, nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]

    return nn.ModuleList(layers)

#请确认执行结果
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [3]:
#创建8层网络的extras模块
def make_extras():
    layers = []
    in_channels = 1024 

    #extras卷积层通道数
    cfg = [256,512,128,256,128,256,128,256]

    layers += [nn.Conv2d(in_channels,cfg[0],kernel_size=1)]
    layers += [nn.Conv2d(cfg[0],cfg[1],kernel_size=3,stride=2,padding=1)]
    layers += [nn.Conv2d(cfg[1],cfg[2],kernel_size=1)]
    layers += [nn.Conv2d(cfg[2],cfg[3],kernel_size=3,stride=2,padding=1)]
    layers += [nn.Conv2d(cfg[3],cfg[4],kernel_size=1)]
    layers += [nn.Conv2d(cfg[4],cfg[5],kernel_size=3)]
    layers += [nn.Conv2d(cfg[5],cfg[6],kernel_size=1)]
    layers += [nn.Conv2d(cfg[6],cfg[7],kernel_size=3)]

    #ReLU函数在正向传播函数中实现
    return nn.ModuleList(layers)

#确认执行结果
extras_test = make_extras()
print(extras_test)





ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


In [ ]:
#loc_layers负责输出DBox的位移值
#创建用于输出DBox的每个分类的置信度confidence的conf_layers

def make_loc_conf(num_classes=21, bbox_aspect_num=[4,6,6,6,4,4]):

    loc_layers = []
    conf_layers = []

    #VGG的第22层，对应source1的卷积层
    loc_layers += [nn.Conv2d(512,bbox_aspect_num[0])]